# Merge Training and Testing Dataset 

In [1]:
import pandas as pd

# Read two CSV files
csv1 = pd.read_csv('train_data.csv')
csv2 = pd.read_csv('same_season_test_data.csv')

# Fill missing 'season' values
# Extract the last 4 characters from 'home_team_season' and 'away_team_season' and convert them to integers
csv2['season'] = csv2['season'].fillna(
    csv2['home_team_season'].str[-4:].astype(float)  # Extract from 'home_team_season' and convert to numeric
)
csv2['season'] = csv2['season'].fillna(
    csv2['away_team_season'].str[-4:].astype(float)  # Extract from 'away_team_season' and convert to numeric
)

# Output rows where 'season' was filled
filled_rows = csv2[csv2['season'].isna()]

# Print the filled rows to console
print(filled_rows)

# Handle rows from csv2 with missing 'season' (fill with 2017)
csv2_missing_season_rows = csv2[csv2['season'].isna()]  # Identify rows in csv2 with missing 'season'
csv2.loc[csv2['season'].isna(), 'season'] = 2017  # Fill missing 'season' with 2017

# Ensure csv1 and csv2 have unique 'id' values
max_id_csv2 = csv2['id'].max()  # Find the maximum 'id' in csv2
csv1['id'] = csv1['id'] + max_id_csv2 + 1  # Adjust csv1 'id' to continue from csv2's 'id'

# Merge the two DataFrames
merged_df = pd.concat([csv2, csv1], axis=0, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_train_test.csv', index=False)

# Count the number of rows for each season
season_counts = merged_df['season'].value_counts().sort_index()

# Save season counts to a CSV file
season_counts.to_csv('season_counts.csv', header=['count'])

# Print season counts
print(season_counts)

        id home_team_abbr away_team_abbr is_night_game home_pitcher  \
1478  1478            GUT            QPO          True    blackma01   

     away_pitcher  home_team_rest  away_team_rest  home_pitcher_rest  \
1478    ucetaed01             1.0             1.0                5.0   

      away_pitcher_rest  ...  away_pitcher_H_batters_faced_skew  \
1478                6.0  ...                          -0.196155   

      away_pitcher_BB_batters_faced_mean  away_pitcher_BB_batters_faced_std  \
1478                           -0.771131                           0.309348   

      away_pitcher_BB_batters_faced_skew  \
1478                            2.267632   

      away_pitcher_leverage_index_avg_mean  \
1478                             -0.022912   

      away_pitcher_leverage_index_avg_std  \
1478                             1.766252   

      away_pitcher_leverage_index_avg_skew  away_pitcher_wpa_def_mean  \
1478                              1.721644                    0.14414   

# Select features we will use

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import skew

train = pd.read_csv("train_data.csv")

# Load the data
file_path = 'merged_train_test.csv'  # Adjust path if necessary
df = pd.read_csv(file_path)

# Step 1: Normalize and restore home_team_wins_mean
# (x-9689)/(2429-9689)
home_mean_max = train.loc[train['id'] == 2429, 'home_team_wins_mean'].iloc[0]
home_mean_min = train.loc[train['id'] == 9689, 'home_team_wins_mean'].iloc[0]
df['home_team_wins_mean_original'] = (df['home_team_wins_mean'] - home_mean_min) / (home_mean_max - home_mean_min)

# Step 2: Normalize and restore home_team_wins_std
home_std_max = train.loc[train['id'] == 6338, 'home_team_wins_std'].iloc[0]
home_std_min = train.loc[train['id'] == 9689, 'home_team_wins_std'].iloc[0]
df['home_team_wins_std_original'] = (df['home_team_wins_std'] - home_std_min) / (2 * (home_std_max - home_std_min))

# Find the value closest to 0
home_shift_skew = train.loc[train['id'] == 4316, 'home_team_wins_skew'].iloc[0]
home_skew_bias_original = skew([0, 0, 1])
home_skew_for_one_thrid = train.loc[train['id'] == 6782, 'home_team_wins_skew'].iloc[0]
df['home_team_wins_skew_original'] = df['home_team_wins_skew'].apply(
    lambda x: ( (x - home_shift_skew) * home_skew_bias_original) / (home_skew_for_one_thrid - home_shift_skew) if pd.notna(x) else np.nan
)

# Step 3: Replace NaN values in home_team_wins_mean_original using home_team_wins_std_original
df['home_std_error'] = df['home_team_wins_std_original'] - (df['home_team_wins_mean_original'] * (1 - df['home_team_wins_mean_original'])) ** 0.5
df['home_skew_error'] = df['home_team_wins_skew_original'] - df['home_team_wins_mean_original'] * (1 - df['home_team_wins_mean_original']) * (1 - 2 * df['home_team_wins_mean_original']) / (df['home_team_wins_std_original'] ** 3)
df['home_mean_error'] = df['home_team_wins_mean_original'] - (1 - df['home_team_wins_skew_original'] * df['home_team_wins_std_original']) / 2
df.loc[df['home_team_wins_mean_original'].isna(), 'home_team_wins_mean_original'] = (
    (1 - df['home_team_wins_skew_original'] * df['home_team_wins_std_original']) / 2
)

# Step 4: Normalize and restore away_team_wins_mean
# (x-2454)/(9689-2454)
away_mean_max = train.loc[train['id'] == 9689, 'away_team_wins_mean'].iloc[0]
away_mean_min = train.loc[train['id'] == 2454, 'away_team_wins_mean'].iloc[0]
df['away_team_wins_mean_original'] = (df['away_team_wins_mean'] - away_mean_min) / (away_mean_max - away_mean_min)

# Step 5: Normalize and restore away_team_wins_std
away_std_max = train.loc[train['id'] == 4316, 'away_team_wins_std'].iloc[0]
away_std_min = train.loc[train['id'] == 9689, 'away_team_wins_std'].iloc[0]
df['away_team_wins_std_original'] = (df['away_team_wins_std'] - away_std_min) / (2 * (away_std_max - away_std_min))

# Find the value closest to 0
away_shift_skew = train.loc[train['id'] == 4316, 'away_team_wins_skew'].iloc[0]
away_skew_bias_original = skew([0, 0, 1])
away_skew_for_one_thrid = train.loc[train['id'] == 1366, 'away_team_wins_skew'].iloc[0]
df['away_team_wins_skew_original'] = df['away_team_wins_skew'].apply(
    lambda x: ( (x - away_shift_skew) * away_skew_bias_original) / (away_skew_for_one_thrid - away_shift_skew) if pd.notna(x) else np.nan
)

# Step 6: Replace NaN values in away_team_wins_mean_original using away_team_wins_std_original
df['away_std_error'] = df['away_team_wins_std_original'] - (df['away_team_wins_mean_original'] * (1 - df['away_team_wins_mean_original'])) ** 0.5
df['away_skew_error'] = df['away_team_wins_skew_original'] - df['away_team_wins_mean_original'] * (1 - df['away_team_wins_mean_original']) * (1 - 2 * df['away_team_wins_mean_original']) / (df['away_team_wins_std_original'] ** 3)
df['away_mean_error'] = df['away_team_wins_mean_original'] - (1 - df['away_team_wins_skew_original'] * df['away_team_wins_std_original']) / 2
df.loc[df['away_team_wins_mean_original'].isna(), 'away_team_wins_mean_original'] = (
    (1 - df['away_team_wins_skew_original'] * df['away_team_wins_std_original']) / 2
)

# (x-2429)/(9689-2429)
home_errors_mean_max = train.loc[train['id'] == 9689, 'home_team_errors_mean'].iloc[0]
home_errors_mean_min = train.loc[train['id'] == 2429, 'home_team_errors_mean'].iloc[0]
df['home_team_errors_mean_original'] = (df['home_team_errors_mean'] - home_errors_mean_min) / (home_errors_mean_max - home_errors_mean_min)

# (x-9296)/(2429-9296)
away_errors_mean_max = train.loc[train['id'] == 2429, 'away_team_errors_mean'].iloc[0]
away_errors_mean_min = train.loc[train['id'] == 9296, 'away_team_errors_mean'].iloc[0]
df['away_team_errors_mean_original'] = (df['away_team_errors_mean'] - away_errors_mean_min) / (away_errors_mean_max - away_errors_mean_min)

# (x-2111)/[(1366-2111)*2]
home_batting_RBI_mean_max = train.loc[train['id'] == 1366, 'home_batting_RBI_mean'].iloc[0]
home_batting_RBI_mean_min = train.loc[train['id'] == 2111, 'home_batting_RBI_mean'].iloc[0]
df['home_batting_RBI_mean_original'] = (df['home_batting_RBI_mean'] - home_batting_RBI_mean_min) / [(home_batting_RBI_mean_max - home_batting_RBI_mean_min) * 2]

# (x-5837)/(9694-5837)
away_batting_RBI_mean_max = train.loc[train['id'] == 9694, 'away_batting_RBI_mean'].iloc[0]
away_batting_RBI_mean_min = train.loc[train['id'] == 5837, 'away_batting_RBI_mean'].iloc[0]
df['away_batting_RBI_mean_original'] = (df['away_batting_RBI_mean'] - away_batting_RBI_mean_min) / (away_batting_RBI_mean_max - away_batting_RBI_mean_min)

# (2 * x - (2429+4909))/(2429-4909)
home_spread_mean_max = train.loc[train['id'] == 2429, 'home_team_spread_mean'].iloc[0]
home_spread_mean_min = train.loc[train['id'] == 4909, 'home_team_spread_mean'].iloc[0]
df['home_team_spread_mean_original'] = (2 * df['home_team_spread_mean'] - (home_spread_mean_max + home_spread_mean_min)) / (home_spread_mean_max - home_spread_mean_min)

# (2 * x - (4909+2454))/(4909-2454)
away_spread_mean_max = train.loc[train['id'] == 4909, 'away_team_spread_mean'].iloc[0]
away_spread_mean_min = train.loc[train['id'] == 2454, 'away_team_spread_mean'].iloc[0]
df['away_team_spread_mean_original'] = (2 * df['away_team_spread_mean'] - (away_spread_mean_max + away_spread_mean_min)) / (away_spread_mean_max - away_spread_mean_min)

# Save the updated DataFrame
df.to_csv(f'data.csv', index=False)

print('home_team_errors_mean_original')
print(df['home_team_errors_mean_original'].mean())
print('away_team_errors_mean_original')
print(df['away_team_errors_mean_original'].mean())
print('home_batting_RBI_mean_original')
print(df['home_batting_RBI_mean_original'].mean())
print('away_batting_RBI_mean_original')
print(df['away_batting_RBI_mean_original'].mean())
print('home_team_spread_mean_original')
print(df['home_team_spread_mean_original'].mean())
print('away_team_spread_mean_original')
print(df['away_team_spread_mean_original'].mean())

/var/folders/rc/_jhlsyv53lj81ztxrdkxlq700000gn/T/ipykernel_57970/1882770783.py:9: DtypeWarning: Columns (165,166) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


home_team_errors_mean_original
0.5667139050088121
away_team_errors_mean_original
0.569671329131434
home_batting_RBI_mean_original
4.292048485361482
away_batting_RBI_mean_original
4.287695434590733
home_team_spread_mean_original
0.014446975360338043
away_team_spread_mean_original
0.00882733283036727


In [3]:
import bisect
import pandas as pd
from math import isqrt
from math import gcd

numerators = range(1, 2000)
denominators = range(1, 162)
fraction_values = []
fraction_pairs = []
for numerator in numerators:
    for denominator in denominators:
        fraction_values.append(numerator / denominator)
        fraction_pairs.append((numerator, denominator))

fraction_values, fraction_pairs = zip(*sorted(zip(fraction_values, fraction_pairs)))

def find_closest_fraction(value):
    odd = value
    value = value % 1 # Ensure the value is positive

    # Perform binary search to find the closest value
    idx = bisect.bisect_left(fraction_values, value)

    # Determine the closest fraction
    if idx == 0:
        closest_fraction = fraction_pairs[0]
    elif idx == len(fraction_values):
        closest_fraction = fraction_pairs[-1]
    else:
        left = fraction_values[idx - 1]
        right = fraction_values[idx]
        closest_fraction = (
            fraction_pairs[idx - 1]
            if abs(value - left) < abs(value - right)
            else fraction_pairs[idx]
        )

    # Extract numerator and denominator
    closest_wins, closest_total_games = closest_fraction[0], closest_fraction[1]

    # Reduce the fraction to its simplest form
    common_divisor = gcd(closest_wins, closest_total_games)
    reduced_wins = closest_wins // common_divisor
    reduced_total_games = closest_total_games // common_divisor
    if reduced_total_games == 161:
        print(f'Old:{odd} , Original value: {value}, Closest fraction: {reduced_wins}/{reduced_total_games}')

    # Return the denominator of the reduced fraction
    return reduced_total_games

df['home_team_errors_mean_den'] = df['home_team_errors_mean_original'].apply(
    lambda x: 1 if pd.isna(x) or abs(x % 1) < 1e-16 else find_closest_fraction(x))
df['away_team_errors_mean_den'] = df['away_team_errors_mean_original'].apply(
    lambda x: 1 if pd.isna(x) or abs(x % 1) < 1e-16 else find_closest_fraction(x))
df['home_batting_RBI_mean_den'] = df['home_batting_RBI_mean_original'].apply(
    lambda x: 1 if pd.isna(x) or abs(x % 1) < 1e-16 else find_closest_fraction(x))
df['away_batting_RBI_mean_den'] = df['away_batting_RBI_mean_original'].apply(
    lambda x: 1 if pd.isna(x) or abs(x % 1) < 1e-16 else find_closest_fraction(x))
df['home_team_spread_mean_den'] = df['home_team_spread_mean_original'].apply(
    lambda x: 1 if pd.isna(x) or abs(x % 1) < 1e-16 else find_closest_fraction(x))
df['away_team_spread_mean_den'] = df['away_team_spread_mean_original'].apply(
    lambda x: 1 if pd.isna(x) or abs(x % 1) < 1e-16 else find_closest_fraction(x))

# 如果原始值是 NaN，則保持分母列為 NaN
df['home_team_errors_mean_den'] = np.where(df['home_team_errors_mean'].isna(), np.nan, df['home_team_errors_mean_den'])
df['away_team_errors_mean_den'] = np.where(df['away_team_errors_mean'].isna(), np.nan, df['away_team_errors_mean_den'])
df['home_batting_RBI_mean_den'] = np.where(df['home_batting_RBI_mean'].isna(), np.nan, df['home_batting_RBI_mean_den'])
df['away_batting_RBI_mean_den'] = np.where(df['away_batting_RBI_mean'].isna(), np.nan, df['away_batting_RBI_mean_den'])
df['home_team_spread_mean_den'] = np.where(df['home_team_spread_mean'].isna(), np.nan, df['home_team_spread_mean_den'])
df['away_team_spread_mean_den'] = np.where(df['away_team_spread_mean'].isna(), np.nan, df['away_team_spread_mean_den'])

# 保存到新的 CSV 文件
df.to_csv(f'data.csv', index=False)

Old:0.5838509316770186 , Original value: 0.5838509316770186, Closest fraction: 94/161
Old:0.45341614906832306 , Original value: 0.45341614906832306, Closest fraction: 73/161
Old:0.6645962732919256 , Original value: 0.6645962732919256, Closest fraction: 107/161
Old:0.49693251533742333 , Original value: 0.49693251533742333, Closest fraction: 80/161
Old:0.5030674846625767 , Original value: 0.5030674846625767, Closest fraction: 81/161
Old:0.596273291925466 , Original value: 0.596273291925466, Closest fraction: 96/161
Old:0.7267080745341615 , Original value: 0.7267080745341615, Closest fraction: 117/161
Old:0.4472049689440995 , Original value: 0.4472049689440995, Closest fraction: 72/161
Old:0.5527950310559008 , Original value: 0.5527950310559008, Closest fraction: 89/161
Old:0.49689440993788825 , Original value: 0.49689440993788825, Closest fraction: 80/161
Old:0.49689440993788825 , Original value: 0.49689440993788825, Closest fraction: 80/161
Old:0.46583850931677023 , Original value: 0.46

# **Calculating** the winner

# Here

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import gcd
from bisect import bisect_left

# Load the data
file_path = 'data.csv'  # adjust path if necessary

df = pd.read_csv(file_path)

# Record the maximum ID value for testing samples based on NaN values in 'home_team_win'
testing_max_id = df[df['home_team_win'].isna()]['id'].max()

# Precompute possible win rates for all combinations of wins and total games
def precompute_win_rates(max_games=200):
    win_rate_list = []
    for total_games in range(1, max_games + 1):
        for wins in range(0, total_games + 1):
            rate = wins / total_games
            win_rate_list.append((rate, wins, total_games))
    # Sort by win rate
    win_rate_list.sort(key=lambda x: x[0])
    return win_rate_list

# Precompute the win rates
precomputed_win_rates = precompute_win_rates()
precomputed_rates = [rate[0] for rate in precomputed_win_rates]

# Define a function to calculate the closest number of wins and total games using binary search
def find_closest_games(winning_rate):
    # Use binary search to find the closest rate
    pos = bisect_left(precomputed_rates, winning_rate)
    if pos == 0:
        closest = precomputed_win_rates[0]
    elif pos == len(precomputed_rates):
        closest = precomputed_win_rates[-1]
    else:
        before = precomputed_win_rates[pos - 1]
        after = precomputed_win_rates[pos]
        closest = before if abs(before[0] - winning_rate) < abs(after[0] - winning_rate) else after

    closest_wins, closest_total_games = closest[1], closest[2]
    # Reduce the ratio of wins and total games
    common_divisor = gcd(closest_wins, closest_total_games)
    reduced_wins = closest_wins // common_divisor
    reduced_total_games = closest_total_games // common_divisor

    return closest_wins, closest_total_games, reduced_wins, reduced_total_games

# Calculate closest wins and total games for each row, for both home and away teams
df[['home_closest_wins', 'home_closest_total_games', 'home_reduced_wins', 'home_reduced_total_games']] = df.apply(
    lambda row: pd.Series(find_closest_games(row['home_team_wins_mean_original'])) if pd.notnull(row['home_team_wins_mean_original']) else pd.Series([np.nan, np.nan, np.nan, np.nan]), axis=1
)

# Calculate closest wins and total games for away team
# Use the restored away_team_wins_mean_original column for calculation
df[['away_closest_wins', 'away_closest_total_games', 'away_reduced_wins', 'away_reduced_total_games']] = df.apply(
    lambda row: pd.Series(find_closest_games(row['away_team_wins_mean_original'])) if pd.notnull(row['away_team_wins_mean_original']) else pd.Series([np.nan, np.nan, np.nan, np.nan]), axis=1
)

# Calculate the estimated win rate and error for home and away teams
def calculate_estimated_rate_and_error(row, original_col, closest_wins_col, closest_total_games_col):
    if pd.notnull(row[original_col]):
        estimated_wins = row[closest_wins_col]
        estimated_total_games = row[closest_total_games_col]
        estimated_rate = estimated_wins / estimated_total_games if estimated_total_games > 0 else np.nan
        error = abs(estimated_rate - row[original_col])
        return pd.Series([estimated_rate, error])
    else:
        return pd.Series([np.nan, np.nan])

# Apply the function to calculate error for home team
df[['home_estimated_win_rate', 'home_rate_error']] = df.apply(
    lambda row: calculate_estimated_rate_and_error(row, 'home_team_wins_mean_original', 'home_closest_wins', 'home_closest_total_games'), axis=1
)

# Apply the function to calculate error for away team
df[['away_estimated_win_rate', 'away_rate_error']] = df.apply(
    lambda row: calculate_estimated_rate_and_error(row, 'away_team_wins_mean_original', 'away_closest_wins', 'away_closest_total_games'), axis=1
)

def consensus(x, y, z):
    if not np.isnan(y) and not np.isnan(z):
        yz = gcd(int(y), int(z))
        if x % yz == 0:
            return x
        elif y % x == 0 and y % z == 0:
            return y
        elif z % x == 0 and z % y == 0:
            return z
        elif yz % x == 0:
            return yz
        elif not np.isnan(x):
            return x
        else:
            return yz
    elif not np.isnan(y):
        if x % y == 0:
            return x
        if y % x == 0:
            return y
        else:
            return x
    elif not np.isnan(z):
        if x % z == 0:
            return x
        if z % x == 0:
            return z
        else:
            return x
    else:
        return x

def lcm(x, y):
    if np.isnan(x):
        return y
    elif np.isnan(y):
        return np.nan
    else:
        num = abs(x * y) // gcd(int(x), int(y))
        if num > 161:
            print(x, y)
            return y
        return num

df['home_consensus_other'] = df.apply(lambda row: consensus(row['home_team_errors_mean_den'], row['home_batting_RBI_mean_den'], row['home_team_spread_mean_den']), axis=1)
df['home_lcm'] = df.apply(lambda row: lcm(row['home_consensus_other'], row['home_reduced_total_games']), axis=1)

df['home_reduced_wins_expanded'] = df.apply(
    lambda row: row['home_reduced_wins'] * (row['home_lcm']  / row['home_reduced_total_games'])
    if not np.isnan(row['home_reduced_total_games']) else np.nan, axis=1)
df['home_reduced_total_games_expanded'] = df['home_lcm'] 

df['away_consensus_other'] = df.apply(lambda row: consensus(row['away_team_errors_mean_den'], row['away_batting_RBI_mean_den'], row['away_team_spread_mean_den']), axis=1)
df['away_lcm'] = df.apply(lambda row: lcm(row['away_consensus_other'], row['away_reduced_total_games']), axis=1)

df['away_reduced_wins_expanded'] = df.apply(
    lambda row: row['away_reduced_wins'] * (row['away_lcm'] / row['away_reduced_total_games'])
    if not np.isnan(row['away_reduced_total_games']) else np.nan, axis=1)
df['away_reduced_total_games_expanded'] = df['away_lcm']


df['home_reduced_wins_original'] = df['home_reduced_wins']
df['home_reduced_total_games_original'] = df['home_reduced_total_games']
df['away_reduced_wins_original'] = df['away_reduced_wins']
df['away_reduced_total_games_original'] = df['away_reduced_total_games']

df['home_reduced_wins'] = df['home_reduced_wins_expanded']
df['home_reduced_total_games'] = df['home_reduced_total_games_expanded']
df['away_reduced_wins'] = df['away_reduced_wins_expanded']
df['away_reduced_total_games'] = df['away_reduced_total_games_expanded']

# Keep only the required columns
columns_to_keep = ['id', 'home_team_abbr', 'away_team_abbr', 'home_team_win', 'season', 
                   'home_team_wins_mean', 'home_team_wins_mean_original',
                   'away_team_wins_mean', 'away_team_wins_mean_original',
                   'home_team_wins_std', 'home_team_wins_std_original',
                   'away_team_wins_std', 'away_team_wins_std_original',
                   'home_std_error', 'away_std_error',
                   'home_team_wins_skew', 'home_team_wins_skew_original',
                   'away_team_wins_skew', 'away_team_wins_skew_original',
                   'home_skew_error', 'away_skew_error',
                   'home_mean_error', 'away_mean_error',
                   'home_reduced_wins', 'home_reduced_total_games',
                   'away_reduced_wins', 'away_reduced_total_games',
                   'away_estimated_win_rate', 'away_rate_error']
df = df[columns_to_keep]

# Save the filtered DataFrame to a new CSV file
filtered_file_path = 'filtered_data.csv'
df.to_csv(filtered_file_path, index=False)

/var/folders/rc/_jhlsyv53lj81ztxrdkxlq700000gn/T/ipykernel_57970/1798093264.py:10: DtypeWarning: Columns (165,166) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


90.0 169.0
140.0 163.0
18.0 162.0
99.0 17.0
124.0 167.0
99.0 166.0
87.0 174.0
13.0 169.0
13.0 169.0
161.0 163.0
81.0 162.0
107.0 173.0
113.0 15.0
13.0 169.0
151.0 3.0
154.0 167.0
83.0 166.0
161.0 163.0
83.0 163.0
95.0 167.0
5.0 175.0
90.0 169.0
138.0 169.0
87.0 174.0
99.0 166.0
144.0 163.0
97.0 59.0
148.0 163.0
138.0 169.0
119.0 167.0
57.0 171.0
83.0 166.0
3.0 168.0
139.0 90.0
15.0 165.0
81.0 162.0
129.0 164.0
81.0 162.0
137.0 14.0
84.0 168.0
119.0 163.0
97.0 164.0
86.0 169.0
137.0 165.0
161.0 185.0
97.0 82.0
1.0 172.0
101.0 162.0
89.0 178.0
83.0 166.0
82.0 164.0
113.0 163.0
89.0 162.0
88.0 176.0
3.0 163.0
137.0 168.0
157.0 162.0
158.0 57.0
118.0 167.0
55.0 165.0
155.0 91.0
141.0 166.0
122.0 59.0
141.0 83.0
94.0 165.0
111.0 164.0
154.0 183.0
119.0 163.0
83.0 82.0
56.0 42.0
99.0 164.0
149.0 175.0
93.0 164.0
83.0 163.0
55.0 165.0
115.0 169.0
140.0 167.0
109.0 167.0
59.0 177.0
85.0 170.0
18.0 162.0
111.0 176.0
3.0 165.0
118.0 167.0
151.0 164.0
55.0 165.0
135.0 163.0
91.0 58.0
158.0 84.0
1

In [5]:
import pandas as pd
import numpy as np
from math import floor, ceil

# 定義處理的函數
def adjust_reduced_games(row):
    home_total = row['home_reduced_total_games']
    away_total = row['away_reduced_total_games']
    home_wins = row['home_reduced_wins']
    away_wins = row['away_reduced_wins']

    if (105 // home_total) + 1 == (160 // home_total):
        multiplier = (160 // home_total)
        row['home_reduced_total_games'] = row['home_reduced_total_games'] * multiplier
        row['home_reduced_wins'] = row['home_reduced_wins'] * multiplier
    
    if (105 // away_total) + 1 == (160 // away_total):
        multiplier = (160 // away_total)
        row['away_reduced_total_games'] = row['away_reduced_total_games'] * multiplier
        row['away_reduced_wins'] = row['away_reduced_wins'] * multiplier    

    # 找出較大值與較小值
    max_games = max(home_total, away_total)
    min_games = min(home_total, away_total)
    thres = 3
    if max_games > 5 and abs(home_total - away_total) > thres and min_games >= 3:
        
        # 計算比值
        ratio = max_games / min_games
        
        # 找到可能的擴分因子，floor 和 ceil
        factors_to_test = [floor(ratio), ceil(ratio)]
        
        # 初始化變量來存儲最佳擴分
        best_scaling_factor = None
        min_difference = float('inf')  # 初始設置為無限大，便於比較
        
        # 測試兩個可能的擴分因子
        for factor in factors_to_test:
            new_min_games = min_games * factor
            
            # 檢查新的擴分後是否符合條件
            if abs(new_min_games - max_games) <= thres:
                # 計算擴分後的差異
                difference = abs(new_min_games - max_games)
                
                # 如果當前的擴分後差異比之前的小，則更新最佳因子
                if difference < min_difference:
                    min_difference = difference
                    best_scaling_factor = factor

        # 如果找到合適的擴分因子，進行擴分
        if best_scaling_factor is not None:
            if min_games == home_total:
                row['home_reduced_total_games'] = round(home_total * best_scaling_factor)
                row['home_reduced_wins'] = round(home_wins * best_scaling_factor)
            elif min_games == away_total:
                row['away_reduced_total_games'] = round(away_total * best_scaling_factor)
                row['away_reduced_wins'] = round(away_wins * best_scaling_factor)

    

    return row

df = df[[id <= testing_max_id for id in df['id']]]

# 應用處理函數到 DataFrame
df = df.apply(adjust_reduced_games, axis=1)

# Save the filtered DataFrame to a new CSV file
filtered_file_path = 'expand_data.csv'
df.to_csv(filtered_file_path, index=False)

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import gcd

# Load the filtered data for further processing
filtered_file_path = 'expand_data.csv'
df = pd.read_csv(filtered_file_path)

# Precompute game sequences for all teams and seasons

def precompute_team_game_sequences(df):
    game_sequences = {}
    for year in df['season'].unique():
        year_data = df[df['season'] == year]
        for team in pd.concat([year_data['home_team_abbr'], year_data['away_team_abbr']]).unique():
            # Find rows where the team is either the home or away team
            home_games = year_data[year_data['home_team_abbr'] == team][['id', 'home_reduced_wins', 'home_reduced_total_games']]
            away_games = year_data[year_data['away_team_abbr'] == team][['id', 'away_reduced_wins', 'away_reduced_total_games']]
            
            # Rename columns for home and away games to have uniform naming
            home_games = home_games.rename(columns={'home_reduced_wins': 'wins', 'home_reduced_total_games': 'games'})
            away_games = away_games.rename(columns={'away_reduced_wins': 'wins', 'away_reduced_total_games': 'games'})
            
            # Concatenate home and away games
            combined_games = pd.concat([home_games, away_games])
            
            # Sort by total games played, with NaN values at the top
            combined_games = combined_games.sort_values(by=['games', 'id'], ascending=[True, True], na_position='first').reset_index(drop=True)
            
            # Fill missing values in the game sequence based on the winning streak or losing streak pattern
            filled_games = fill_missing_game_sequence(combined_games)
            
            # Store the game sequence
            game_sequences[(team, year)] = list(filled_games.itertuples(index=False, name=None))
    return game_sequences

# Helper function to fill missing game sequences
# Also tries to reduce and match any new data to existing rows
# If found, replace and delete the original data from the sequence
def fill_missing_game_sequence(combined_games):
    filled_games = combined_games.copy()
    games_list = filled_games.to_records(index=False)
    i = len(games_list) - 2
    print(combined_games)
    while i >= 0:
        print(i)
        # Skip if games_list[i].id is NaN
        if pd.isna(games_list[i].id) or pd.isna(games_list[i].wins):
            i -= 1
            continue
        # Check if the current games_list[i] has a label for home_team_win
        current_id = games_list[i].id
        # Find the corresponding row in the original df to check home_team_win
        current_row = df[df['id'] == current_id]
        if not current_row.empty and pd.notna(current_row.iloc[0]['home_team_win']):
            if current_row.iloc[0]['home_team_win'] == True:
                # Add a new winning data point
                new_wins = int(games_list[i].wins - 1)
                new_games = int(games_list[i].games - 1)
            elif current_row.iloc[0]['home_team_win'] == False:
                # Add a new losing data point
                new_wins = int(games_list[i].wins)
                new_games = int(games_list[i].games - 1)
            # Check if this point already exists
            reduced_wins = new_wins // gcd(new_wins, new_games) if new_games > 0 else new_wins
            reduced_games = new_games // gcd(new_wins, new_games) if new_games > 0 else new_games
            match = filled_games[(filled_games['wins'] == reduced_wins) & (filled_games['games'] == reduced_games)]
            if not match.empty:
                matched_id = match.iloc[0]['id']
                matched_index = match.index[0]
            else:
                matched_id = filled_games['id'][0]
                matched_index = 0
            # Append a new row
            new_row = pd.DataFrame({'id': [matched_id], 'wins': [new_wins], 'games': [new_games]})
            filled_games = pd.concat([filled_games, new_row], ignore_index=True)
            # Drop the original matched row
            filled_games = filled_games.drop(matched_index).reset_index(drop=True)
            # Sort by 'games' column and then by 'id' (ascending order), with NaN values at the beginning
            filled_games = filled_games.sort_values(by=['games', 'id'], ascending=[True, True], na_position='first').reset_index(drop=True)
            
            i -= 1
            continue
        head = i
        tail = i + 1
        if tail == len(games_list) - 1 or games_list[tail+1].games != games_list[tail].games:
            if head == 0 or games_list[head-1].games != games_list[head].games:
                if not np.isnan(games_list[head].games) and not np.isnan(games_list[tail].games) and games_list[head].games != games_list[tail].games:
                    if games_list[tail].wins - games_list[head].wins == games_list[tail].games - games_list[head].games:
                        # winning streak
                        # Fill in the missing games as wins (winning streak)
                        missing_count = int(games_list[tail].games - games_list[head].games)
                        for j in range(1, missing_count):
                            new_wins = int(games_list[head].wins + j)
                            new_games = int(games_list[head].games + j)
                            reduced_wins = new_wins // gcd(new_wins, new_games)
                            reduced_games = new_games // gcd(new_wins, new_games)
                            match = filled_games[(filled_games['wins'] == reduced_wins) & (filled_games['games'] == reduced_games)]
                            if not match.empty:
                                matched_id = match.iloc[0]['id']
                                matched_index = match.index[0]
                            else:
                                matched_id = filled_games['id'][0]
                                matched_index = 0
                            # 1. Append a new row
                            new_row = pd.DataFrame({'id': [matched_id], 'wins': [new_wins], 'games': [new_games]})
                            filled_games = pd.concat([filled_games, new_row], ignore_index=True)
                            # 2. Drop a specific row (假設 match.index[0] 是你要刪除的列)
                            filled_games = filled_games.drop(matched_index).reset_index(drop=True)
                            # 3. Sort by 'games' column and then by 'id' (ascending order), with NaN values at the beginning
                            filled_games = filled_games.sort_values(by=['games', 'id'], ascending=[True, True], na_position='first').reset_index(drop=True)
                    elif games_list[tail].wins == games_list[head].wins:
                        # Fill in the missing games as losses (losing streak)
                        missing_count = int(games_list[tail].games - games_list[head].games)
                        for j in range(1, missing_count):
                            new_wins = int(games_list[head].wins)
                            new_games = int(games_list[head].games + j)
                            reduced_wins = new_wins // gcd(new_wins, new_games)
                            reduced_games = new_games // gcd(new_wins, new_games)
                            match = filled_games[(filled_games['wins'] == reduced_wins) & (filled_games['games'] == reduced_games)]
                            if not match.empty:
                                matched_id = match.iloc[0]['id']
                                matched_index = match.index[0]
                            else:
                                matched_id = filled_games['id'][0]
                                matched_index = 0
                            # 1. Append a new row
                            new_row = pd.DataFrame({'id': [matched_id], 'wins': [new_wins], 'games': [new_games]})
                            filled_games = pd.concat([filled_games, new_row], ignore_index=True)
                            # 2. Drop a specific row (假設 match.index[0] 是你要刪除的列)
                            filled_games = filled_games.drop(matched_index).reset_index(drop=True)
                            # 3. Sort by 'games' column and then by 'id' (ascending order), with NaN values at the beginning
                            filled_games = filled_games.sort_values(by=['games', 'id'], ascending=[True, True], na_position='first').reset_index(drop=True)
        i -= 1
    print(filled_games)
    return filled_games

# Precompute all game sequences
game_sequences = precompute_team_game_sequences(df)

# Define a function to get the precomputed game sequence
def get_team_game_sequence(team, year):
    return game_sequences.get((team, year), [])

# Fill missing values for testing data in 'home_team_win'
for index, row in df[df['id'] <= testing_max_id].iterrows():
    home_team = row['home_team_abbr']
    away_team = row['away_team_abbr']
    year = row['season']
    game_id = row['id']
    
    if year == 0:
        df.at[index, 'home_team_win'] = row['home_team_wins_mean_original'] > row['away_team_wins_mean_original']
        continue
    
    # Get sequences for home and away teams
    home_sequence = get_team_game_sequence(home_team, year)
    away_sequence = get_team_game_sequence(away_team, year)

    # Find the relevant game information for the current testing data id
    home_game_data = next((item for item in home_sequence if item[0] == game_id), None)
    away_game_data = next((item for item in away_sequence if item[0] == game_id), None)
    
    # Helper function to calculate win rate from previous group
    def calculate_win_rate_from_previous(group, current_wins, current_games):
        future_group = [g for g in group if g[2] != np.nan and g[2] > current_games]
        if not future_group:
            return np.nan
        
        closest_games = min([g[2] for g in future_group])  # Find the closest but smaller game count
        group_with_closest_games = [g for g in future_group if g[2] == closest_games]  # Get all entries with that closest game count
        avg_wins = np.mean([g[1] for g in group_with_closest_games])
        return (avg_wins - current_wins) / (closest_games - current_games)

    # Calculate win rate for home and away teams if data is available
    home_win_rate = calculate_win_rate_from_previous(home_sequence, home_game_data[1], home_game_data[2]) if home_game_data else np.nan
    away_win_rate = calculate_win_rate_from_previous(away_sequence, away_game_data[1], away_game_data[2]) if away_game_data else np.nan

    # Determine the correct prediction
    if home_win_rate in [0, 1]:
        df.at[index, 'home_team_win'] = (home_win_rate == 1)
    elif away_win_rate in [0, 1]:
        df.at[index, 'home_team_win'] = (away_win_rate == 0)
    elif pd.notnull(home_win_rate) and home_win_rate != 0.5:
        df.at[index, 'home_team_win'] = (home_win_rate > 0.5)
    elif pd.notnull(away_win_rate) and away_win_rate != 0.5:
        df.at[index, 'home_team_win'] = (away_win_rate < 0.5)
    else:
        df.at[index, 'home_team_win'] = row['home_team_wins_mean_original'] > row['away_team_wins_mean_original']

# Generate timestamp for file names
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save the updated DataFrame
df.to_csv(f'processed_data_{timestamp}.csv', index=False)

# Save testing sample IDs and home_team_win values
testing_sample = df[df['id'] <= testing_max_id][['id', 'home_team_win']]
testing_sample.to_csv(f'submit_{timestamp}.csv', index=False)

testing_sample['home_team_win'] = testing_sample['home_team_win'].apply(lambda x: not x)
testing_sample.to_csv(f'submit_flip_{timestamp}.csv', index=False)

      id   wins  games
0   2439   53.0   84.0
1   4345   54.0   85.0
2   1290   54.0   86.0
3   2398   69.0  105.0
4    924   69.0  106.0
..   ...    ...    ...
70  5278  109.0  174.0
71   127  110.0  175.0
72  3798  110.0  176.0
73   379  112.0  178.0
74  2751  112.0  179.0

[75 rows x 3 columns]
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
      id   wins  games
0   1290   54.0   86.0
1   2398   69.0  105.0
2    924   69.0  106.0
3   5162   69.0  107.0
4    313   69.0  108.0
..   ...    ...    ...
70   127  110.0  175.0
71  3798  110.0  176.0
72  2439  111.0  177.0
73   379  112.0  178.0
74  2751  112.0  179.0

[75 rows x 3 columns]
      id  wins  games
0   1922  50.0  105.0
1   1256  50.0  106.0
2   3628  51.0  107.0
3   3392  52.0  108.0
4   2410  52.0  109.0
5   4029  53.0  110.0
6   1083  53.0  111.0
7   5856  53.0

/var/folders/rc/_jhlsyv53lj81ztxrdkxlq700000gn/T/ipykernel_57970/2307045004.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'home_team_win'] = (home_win_rate == 1)
